In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime import DeepSort
from collections import deque

model_car = YOLO("yolov8n_car.pt")
model_traffic = YOLO("traffic.pt")
model_seg = YOLO("yolov8s-seg_stop_line.pt")
tracker = DeepSort(max_age=30)
traffic_light_buffer = deque(maxlen=10)

def match_car_to_traffic_light(car_bbox, traffic_lights):
    car_center = [(car_bbox[0] + car_bbox[2])/2, (car_bbox[1] + car_bbox[3])/2]
    min_dist = float("inf")
    closest_light = None
    for light in traffic_lights:
        light_center = [(light[0] + light[2])/2, (light[1] + light[3])/2]
        distance = np.sqrt((car_center[0]-light_center[0])**2 + (car_center[1]-light_center[1])**2)
        if distance < min_dist:
            min_dist = distance
            closest_light = light
    return closest_light

def is_crossing_stop_line(car_bbox, stop_line_mask):
    x_min, y_min, x_max, y_max = car_bbox
    h, w = stop_line_mask.shape
    pixel_x = int(((x_min + x_max) / 2) * w)
    pixel_y = int(y_max * h)
    return stop_line_mask[pixel_y, pixel_x] == 1 if 0 <= pixel_x < w and 0 <= pixel_y < h else False

cap = cv2.VideoCapture("input_video.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    car_results = model_car(frame)
    car_boxes = car_results[0].boxes.xyxy.cpu().numpy()
    car_confidences = car_results[0].boxes.conf.cpu().numpy()
    tracks = tracker.update(car_boxes, car_confidences, frame)

    traffic_results = model_traffic(frame)
    traffic_lights = []
    for box, cls in zip(traffic_results[0].boxes.xyxy.cpu().numpy(), traffic_results[0].boxes.cls.cpu().numpy()):
        traffic_lights.append({"bbox": box, "signal": model_traffic.names[int(cls)]})

    seg_results = model_seg(frame)
    stop_line_mask = seg_results[0].masks.data[0].cpu().numpy() if seg_results[0].masks else np.zeros(frame.shape[:2])

    current_signals = [light["signal"] for light in traffic_lights]
    traffic_light_buffer.extend(current_signals)

    for track in tracks:
        track_id = track.track_id
        car_bbox = track.to_tlbr()
        closest_light = match_car_to_traffic_light(car_bbox, [light["bbox"] for light in traffic_lights])
        
        if closest_light and traffic_lights[[light["bbox"] for light in traffic_lights].index(closest_light)]["signal"] == "red":
            if traffic_light_buffer.count("red") >= 5:
                if is_crossing_stop_line(car_bbox, stop_line_mask):
                    print(f"Violation by car {track_id}")

cap.release()